In [17]:
import pygame
import numpy as np
from scipy.linalg import cholesky

# UnscentedKalmanFilter 클래스는 이전과 동일하게 재사용됩니다.
class UnscentedKalmanFilter:
    def __init__(self, x0, P, Q, R, dt, fx, hx):
        self.x = np.array(x0)
        self.P = np.array(P)
        self.Q = np.array(Q)
        self.R = np.array(R)
        self.dt = dt
        self.fx = fx
        self.hx = hx
        self.n = len(x0)
        self.m = self.R.shape[0]
        alpha, beta, kappa = 1e-3, 2, 0
        self.lambda_ = alpha**2 * (self.n + kappa) - self.n
        self.Wm = np.full(2 * self.n + 1, 0.5 / (self.n + self.lambda_))
        self.Wc = np.full(2 * self.n + 1, 0.5 / (self.n + self.lambda_))
        self.Wm[0] = self.lambda_ / (self.n + self.lambda_)
        self.Wc[0] = self.Wm[0] + (1 - alpha**2 + beta)

    def _generate_sigma_points(self):
        sigma_points = np.zeros((2 * self.n + 1, self.n))
        try: U = cholesky((self.n + self.lambda_) * self.P)
        except np.linalg.LinAlgError: U = np.zeros_like(self.P)
        sigma_points[0] = self.x
        for i in range(self.n):
            sigma_points[i + 1] = self.x + U[i]
            sigma_points[i + self.n + 1] = self.x - U[i]
        return sigma_points

    def predict(self, u):
        sigma_points = self._generate_sigma_points()
        self.sigma_points_pred = np.array([self.fx(sp, u, self.dt) for sp in sigma_points])
        self.x_pred = np.dot(self.Wm, self.sigma_points_pred)
        P_pred = np.zeros((self.n, self.n))
        for i in range(2 * self.n + 1):
            y = self.sigma_points_pred[i] - self.x_pred
            P_pred += self.Wc[i] * np.outer(y, y)
        self.P_pred = P_pred + self.Q
        self.x = self.x_pred
        self.P = self.P_pred

    def update(self, z):
        sigma_points_meas = np.array([self.hx(sp) for sp in self.sigma_points_pred])
        z_pred = np.dot(self.Wm, sigma_points_meas)
        P_zz = np.zeros((self.m, self.m))
        P_xz = np.zeros((self.n, self.m))
        for i in range(2 * self.n + 1):
            res_z = (sigma_points_meas[i] - z_pred).reshape(self.m, 1)
            res_x = (self.sigma_points_pred[i] - self.x_pred).reshape(self.n, 1)
            P_zz += self.Wc[i] * res_z @ res_z.T
            P_xz += self.Wc[i] * res_x @ res_z.T
        P_zz += self.R
        K = P_xz @ np.linalg.inv(P_zz)
        innovation = (z - z_pred).reshape(self.m, 1)
        self.x = self.x_pred + (K @ innovation).flatten()
        self.P = self.P_pred - K @ P_zz @ K.T

# --- 시스템 모델 정의 ---
# 상태: [angle, bias], 제어입력 u: 자이로 측정값(각속도)
def state_transition_function(x, u, dt):
    angle, bias = x
    # 예측 모델: 현재 각도 + (자이로값 - 예측된 바이어스) * 시간
    angle_next = angle + (u - bias) * dt
    # 바이어스는 거의 변하지 않는다고 가정
    bias_next = bias
    return np.array([angle_next, bias_next])

# 측정 모델: 가속도계는 상태 중 각도(angle)만 측정 가능
def measurement_function(x):
    return np.array([x[0]])

# --- Pygame 및 시뮬레이션 파라미터 ---
pygame.init()

# 2. 그 다음에 폰트를 포함한 다른 Pygame 기능을 사용합니다.
# try:
#     font = pygame.font.Font("NanumGothic.ttf", 30)
# except pygame.error as e:
#     print(f"폰트 로딩 에러: {e}")
#     print("NanumGothic.ttf 파일이 없거나, Pygame 초기화에 문제가 있습니다.")
#     # 이 라인도 pygame.init() 뒤에 와야 정상 작동합니다.
#     font = pygame.font.Font(None, 30) 

# ... import 문들 아래에 추가 ...


font = pygame.font.SysFont("Malgun Gothic", 30)
WIDTH, HEIGHT = 1000, 800
screen = pygame.display.set_mode((WIDTH, HEIGHT))
pygame.display.set_caption("UKF 센서 융합 (자이로+가속도계)")
clock = pygame.time.Clock()

dt = 0.05
num_steps = 1000

# 노이즈 파라미터
gyro_noise_std = 0.5    # 자이로 측정 노이즈
accel_noise_std = 3.0   # 가속도계 측정 노이즈
gyro_bias_std = 0.01    # 자이로 바이어스의 실제 변화 정도
gyro_bias_init = 2.0    # 실제 자이로 바이어스 초기값

# 3x3 프로세스 노이즈 행렬 Q (자이로 불확실성)
Q = np.eye(2) * gyro_noise_std**2
# 2x2 측정 노이즈 행렬 R (가속도계 불확실성)
R = np.eye(1) * accel_noise_std**2

# ==============================================================
# 단계 1: 가상 센서 데이터 생성
# ==============================================================
print("단계 1: 가상 센서 데이터 생성 중...")
true_angles = []
gyro_inputs = []
accel_measurements = []

true_bias = gyro_bias_init
for k in range(num_steps):
    # 실제 각도는 sin 함수를 따라 움직인다고 가정
    true_angle_rad = np.sin(k * 0.02) * (np.pi / 4) # +/- 45도 사이
    true_angle_deg = np.rad2deg(true_angle_rad)
    true_angles.append(true_angle_deg)
    
    # 실제 각속도 (각도를 시간에 대해 미분)
    true_angular_velocity = 0.02 * (np.pi / 4) * np.cos(k * 0.02)
    true_angular_velocity_deg = np.rad2deg(true_angular_velocity)
    
    # 바이어스는 매우 느리게 변화 (Random Walk)
    true_bias += np.random.normal(0, gyro_bias_std)
    
    # 노이즈를 포함한 자이로 측정값 생성 (제어 입력 u로 사용됨)
    gyro_reading = true_angular_velocity_deg + true_bias + np.random.normal(0, gyro_noise_std)
    gyro_inputs.append(gyro_reading)
    
    # 노이즈를 포함한 가속도계 측정값 생성 (측정값 z로 사용됨)
    accel_reading = true_angle_deg + np.random.normal(0, accel_noise_std)
    accel_measurements.append(accel_reading)

# ==============================================================
# 단계 2: UKF 적용
# ==============================================================
print("단계 2: UKF 적용 중...")
# 초기 추정값: 각도 0, 바이어스 0으로 가정
x_est_initial = np.array([0.0, 0.0])
# 초기 공분산: 불확실성을 크게 설정
P_initial = np.eye(2) * 100

ukf = UnscentedKalmanFilter(
    x0=x_est_initial, P=P_initial, Q=Q, R=R, dt=dt,
    fx=state_transition_function,
    hx=measurement_function
)

estimated_states = []
for k in range(num_steps):
    # 예측 단계: 자이로 측정값을 제어 입력으로 사용
    ukf.predict(u=gyro_inputs[k])
    # 업데이트 단계: 가속도계 측정값을 사용
    ukf.update(np.array([accel_measurements[k]]))
    estimated_states.append(ukf.x.copy())

# ==============================================================
# 단계 3: 시각화
# ==============================================================
print("단계 3: 시각화 시작...")
running = True
frame_index = 0

def draw_bar(surface, y_pos, angle, text, color, value_text):
    bar_width, bar_height = 300, 20
    base_bar = pygame.Surface((bar_width, bar_height), pygame.SRCALPHA)
    base_bar.fill(color)
    
    rotated_bar = pygame.transform.rotate(base_bar, -angle)
    rect = rotated_bar.get_rect(center=(WIDTH/2, y_pos))
    surface.blit(rotated_bar, rect)
    
    # 이 함수 안에서는 전역으로 선언된 font 객체를 사용합니다.
    label = font.render(text, True, (0,0,0)) # pygame.font.Font(None, 30) 대신 font 사용
    surface.blit(label, (50, y_pos - 10))

    val_label = font.render(value_text, True, (50,50,50)) # pygame.font.Font(None, 30) 대신 font 사용
    surface.blit(val_label, (WIDTH - 250, y_pos - 10))


while running:
    for event in pygame.event.get():
        if event.type == pygame.QUIT: running = False

    screen.fill((220, 220, 220))

    if frame_index < num_steps:
        # 각도 값 가져오기
        true_angle = true_angles[frame_index]
        accel_angle = accel_measurements[frame_index]
        ukf_state = estimated_states[frame_index]
        ukf_angle = ukf_state[0]
        ukf_bias = ukf_state[1]

        # 3개의 막대를 그려 각도 비교
        draw_bar(screen, 200, accel_angle, "가속도계 측정 각도:", (255, 100, 100), f"{accel_angle:.1f}°")
        draw_bar(screen, 400, true_angle, "실제 각도:", (100, 100, 255), f"{true_angle:.1f}°")
        draw_bar(screen, 600, ukf_angle, "UKF 추정 각도:", (100, 255, 100), f"{ukf_angle:.1f}°")

        # UKF가 추정한 자이로 바이어스 값 표시
        bias_text = font.render(f"추정된 자이로 바이어스: {ukf_bias:.2f}", True, (0,0,0)) # font 객체 사용
        screen.blit(bias_text, (50, 700))
        
        frame_index += 1
    else:
        # 완료 메시지를 위한 폰트 크기만 따로 설정
        text_done = font.render("시뮬레이션 완료", True, (0, 100, 0))
        text_rect = text_done.get_rect(center=(WIDTH // 2, HEIGHT // 2))
        screen.blit(text_done, text_rect)

    pygame.display.flip()
    clock.tick(60)

pygame.quit()
print("시각화 종료 및 Pygame 종료됨.")

단계 1: 가상 센서 데이터 생성 중...
단계 2: UKF 적용 중...
단계 3: 시각화 시작...
시각화 종료 및 Pygame 종료됨.


In [6]:
import pygame
import numpy as np
from scipy.linalg import cholesky

# UnscentedKalmanFilter 클래스는 변경 없이 그대로 사용합니다.
class UnscentedKalmanFilter:
    def __init__(self, x0, P, Q, R, dt, fx, hx):
        self.x = np.array(x0)
        self.P = np.array(P)
        self.Q = np.array(Q)
        self.R = np.array(R)
        self.dt = dt
        self.fx = fx
        self.hx = hx
        self.n = len(x0)
        self.m = self.R.shape[0]
        alpha, beta, kappa = 1e-3, 2, 0
        self.lambda_ = alpha**2 * (self.n + kappa) - self.n
        self.Wm = np.full(2 * self.n + 1, 0.5 / (self.n + self.lambda_))
        self.Wc = np.full(2 * self.n + 1, 0.5 / (self.n + self.lambda_))
        self.Wm[0] = self.lambda_ / (self.n + self.lambda_)
        self.Wc[0] = self.Wm[0] + (1 - alpha**2 + beta)

    def _generate_sigma_points(self):
        sigma_points = np.zeros((2 * self.n + 1, self.n))
        try: U = cholesky((self.n + self.lambda_) * self.P)
        except np.linalg.LinAlgError: U = np.zeros_like(self.P)
        sigma_points[0] = self.x
        for i in range(self.n):
            sigma_points[i + 1] = self.x + U[i]
            sigma_points[i + self.n + 1] = self.x - U[i]
        return sigma_points

    def predict(self, u):
        sigma_points = self._generate_sigma_points()
        self.sigma_points_pred = np.array([self.fx(sp, u, self.dt) for sp in sigma_points])
        self.x_pred = np.dot(self.Wm, self.sigma_points_pred)
        P_pred = np.zeros((self.n, self.n))
        for i in range(2 * self.n + 1):
            y = self.sigma_points_pred[i] - self.x_pred
            P_pred += self.Wc[i] * np.outer(y, y)
        self.P_pred = P_pred + self.Q
        self.x = self.x_pred
        self.P = self.P_pred

    def update(self, z):
        sigma_points_meas = np.array([self.hx(sp) for sp in self.sigma_points_pred])
        z_pred = np.dot(self.Wm, sigma_points_meas)
        P_zz = np.zeros((self.m, self.m))
        P_xz = np.zeros((self.n, self.m))
        for i in range(2 * self.n + 1):
            res_z = (sigma_points_meas[i] - z_pred).reshape(self.m, 1)
            res_x = (self.sigma_points_pred[i] - self.x_pred).reshape(self.n, 1)
            P_zz += self.Wc[i] * res_z @ res_z.T
            P_xz += self.Wc[i] * res_x @ res_z.T
        P_zz += self.R
        K = P_xz @ np.linalg.inv(P_zz)
        innovation = (z - z_pred).reshape(self.m, 1)
        self.x = self.x_pred + (K @ innovation).flatten()
        self.P = self.P_pred - K @ P_zz @ K.T

# 3D 시스템 모델 정의 (변경 없음)
def state_transition_function(x, u, dt):
    phi, theta, psi = x; p, q, r = u
    epsilon = 1e-8; cos_theta = np.cos(theta)
    if abs(cos_theta) < epsilon: cos_theta = epsilon
    phi_dot = p + np.sin(phi) * np.tan(theta) * q + np.cos(phi) * np.tan(theta) * r
    theta_dot = np.cos(phi) * q - np.sin(phi) * r
    psi_dot = (np.sin(phi) / cos_theta) * q + (np.cos(phi) / cos_theta) * r
    return x + np.array([phi_dot, theta_dot, psi_dot]) * dt
def measurement_function(x):
    return np.array([x[0], x[1]])

# --- Pygame 및 시뮬레이션 파라미터 ---
pygame.init()
WIDTH, HEIGHT = 1200, 800  # 3개 큐브를 위해 너비 확장
screen = pygame.display.set_mode((WIDTH, HEIGHT))
pygame.display.set_caption("3D UKF 시뮬레이션 (실제 vs 측정 vs 추정)")
clock = pygame.time.Clock()
font = pygame.font.SysFont("Malgun Gothic", 28)
title_font = pygame.font.SysFont("Malgun Gothic", 32, bold=True)


dt = 0.05
num_steps = 1000
gyro_noise_std = 0.05
accel_noise_std = 0.1
Q = np.eye(3) * gyro_noise_std**2
R = np.eye(2) * accel_noise_std**2

# 3D 큐브 시각화 함수 (변경 없음)
def draw_cube(surface, angles_rad, center_pos, size, color, width):
    phi, theta, psi = angles_rad
    points = np.array([[-1,-1,-1],[1,-1,-1],[1,1,-1],[-1,1,-1],[-1,-1,1],[1,-1,1],[1,1,1],[-1,1,1]])*size
    Rx = np.array([[1,0,0],[0,np.cos(phi),-np.sin(phi)],[0,np.sin(phi),np.cos(phi)]])
    Ry = np.array([[np.cos(theta),0,np.sin(theta)],[0,1,0],[-np.sin(theta),0,1]])
    Rz = np.array([[np.cos(psi),-np.sin(psi),0],[np.sin(psi),np.cos(psi),0],[0,0,1]])
    rotated_points = (Rz @ Ry @ Rx @ points.T).T
    projected_points = rotated_points[:, :2] + np.array(center_pos)
    edges = [[0,1],[1,2],[2,3],[3,0],[4,5],[5,6],[6,7],[7,4],[0,4],[1,5],[2,6],[3,7]]
    for edge in edges:
        pygame.draw.line(surface, color, projected_points[edge[0]], projected_points[edge[1]], width)

# 단계 1, 2 (데이터 생성 및 UKF 적용)는 이전 코드와 동일합니다.
# ==============================================================
# 단계 1: 가상 3D 데이터 생성
# ==============================================================
true_states = []; gyro_inputs = []; accel_measurements = []
def get_true_gyro(t):
    return np.array([0.1*np.sin(t*0.5), 0.5*np.cos(t*0.3), 0.3*np.sin(t*0.2)])
current_true_state = np.zeros(3)
for k in range(num_steps):
    t = k * dt
    true_gyro = get_true_gyro(t)
    current_true_state = state_transition_function(current_true_state, true_gyro, dt)
    true_states.append(current_true_state.copy())
    gyro_reading = true_gyro + np.random.normal(0, gyro_noise_std, 3)
    gyro_inputs.append(gyro_reading)
    accel_reading = measurement_function(current_true_state) + np.random.normal(0, accel_noise_std, 2)
    accel_measurements.append(accel_reading)
# ==============================================================
# 단계 2: UKF 적용
# ==============================================================
x_est_initial = np.array([0.1, -0.1, 0.05]); P_initial = np.eye(3) * 0.1
ukf = UnscentedKalmanFilter(x_est_initial, P_initial, Q, R, dt, state_transition_function, measurement_function)
estimated_states = []
for k in range(num_steps):
    ukf.predict(u=gyro_inputs[k]); ukf.update(z=accel_measurements[k])
    estimated_states.append(ukf.x.copy())

# ==============================================================
# 단계 3: 시각화 (측정 큐브 추가)
# ==============================================================
print("단계 3: 시각화 시작...")
running = True
frame_index = 0

while running:
    for event in pygame.event.get():
        if event.type == pygame.QUIT: running = False
    screen.fill((20, 20, 40))

    if frame_index < num_steps:
        # --- 데이터 가져오기 ---
        true_angles = true_states[frame_index]
        est_angles = estimated_states[frame_index]
        meas_roll, meas_pitch = accel_measurements[frame_index]

        # --- 시각화를 위한 측정 자세 벡터 생성 ---
        # Yaw는 측정값이 없으므로, UKF의 추정 Yaw 값을 빌려와서 사용
        est_yaw = est_angles[2]
        meas_angles_vis = np.array([meas_roll, meas_pitch, est_yaw])

        # --- 3개의 큐브 그리기 ---
        cube_y_pos = HEIGHT * 0.4
        draw_cube(screen, true_angles,     (WIDTH * 0.2, cube_y_pos), 100, (100, 150, 255), 3)
        draw_cube(screen, meas_angles_vis, (WIDTH * 0.5, cube_y_pos), 100, (255, 180, 100), 2)
        draw_cube(screen, est_angles,      (WIDTH * 0.8, cube_y_pos), 100, (100, 255, 150), 3)

        # --- 타이틀 텍스트 ---
        def draw_title(text, x_pos, y_pos):
            label = title_font.render(text, True, (255, 255, 255))
            label_rect = label.get_rect(center=(x_pos, y_pos))
            screen.blit(label, label_rect)

        title_y_pos = HEIGHT * 0.1
        draw_title("실제 자세 (True)", WIDTH * 0.2, title_y_pos)
        draw_title("측정 기반 자세 (Measurement)", WIDTH * 0.5, title_y_pos)
        draw_title("UKF 추정 자세 (Estimate)", WIDTH * 0.8, title_y_pos)

        # --- 상세 정보 텍스트 ---
        def draw_detail_text(x_pos, y_start, angles):
            texts = [f"Roll (φ): {np.rad2deg(angles[0]):.1f}°",
                     f"Pitch (θ): {np.rad2deg(angles[1]):.1f}°",
                     f"Yaw (ψ): {np.rad2deg(angles[2]):.1f}°"]
            for i, text in enumerate(texts):
                label = font.render(text, True, (220, 220, 220))
                screen.blit(label, (x_pos, y_start + i * 35))

        detail_y_pos = HEIGHT * 0.65
        draw_detail_text(WIDTH * 0.2 - 100, detail_y_pos, true_angles)
        draw_detail_text(WIDTH * 0.5 - 100, detail_y_pos, meas_angles_vis)
        draw_detail_text(WIDTH * 0.8 - 100, detail_y_pos, est_angles)
        
        frame_index += 1
    else:
        # 완료 메시지
        font_done = pygame.font.SysFont("Malgun Gothic", 60, bold=True)
        text_done = font_done.render("시뮬레이션 완료", True, (100, 255, 150))
        screen.blit(text_done, text_done.get_rect(center=(WIDTH/2, HEIGHT/2)))

    pygame.display.flip()
    clock.tick(60)

pygame.quit()

단계 3: 시각화 시작...
